In [ ]:
import os
import pandas as pd

# get all files starting with "xval_" in results
#files = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and "xval_2_5" in f or "xval_3_5" in f and "old" not in f]
files_6_op = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and ("xval_fad_big_4_5" in f or "xval_fad_big_5_5" in f) and ("mlm" not in f)]
files_4_op = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and (("xval_fad_big_3_5" in f) and ("mlm" not in f)) or ("xval_fad_normal_8_5" in f)]
files_4_op_mlm = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and "mlm_xval_fad_big_4_5" in f]
files_4_op_small_model = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and "xval_fad_big_small_model_5_5" in f]
files_4_op_uniform = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and ("xval_fad_big_small_model_6_5" in f or "xval_fad_big_small_model_7_5" in f or "xval_fad_uniform_7_5" in f)]
files_unknown = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and ("xval_fad_uniform_3_20" in f) or ("xval_fad_uniform_small_lengthscale_21_5" in f) or ("xval_orbits_varying_lengthscale_21_5" in f) or ("xval_orbits_varying_lengthscale_20_5" in f)]
#files_unknown = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and ("xval_orbits_varying_lengthscale_21_5" in f) or ("xval_orbits_varying_lengthscale_20_5" in f)]
files_varying_lengthscale = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and ("xval_fad_normal_8_5" in f) or ("xval_fad_uniform_small_lengthscale_9_5" in f) or ("xval_fad_uniform_small_lengthscale_10_5" in f) or ("xval_fad_uniform_small_lengthscale_13_5" in f) or ("xval_fad_uniform_13_5" in f) or ("xval_fad_uniform_3_20" in f)] + files_4_op_uniform + files_4_op
files_different_depth = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and ("xval_fad_uniform_13_5" in f) or ("xval_fad_uniform_14_5" in f)]
#files_4_op_uniform_3 = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and ("xval_fad_uniform_7_5" in f)]
#files_4_op_uniform_jz = [f for f in os.listdir("results") if os.path.isfile(os.path.join("results", f)) and ("xval_fad_big_6_5" in f)]
print(f"Found {len(files_6_op)} files")
print(f"Found {len(files_4_op)} files")
print(f"Found {len(files_4_op_mlm)} files")
print(f"Found {len(files_4_op_small_model)} files")
print(f"Found {len(files_4_op_uniform)} files")
print(f"Found {len(files_unknown)} files")
print(f"Found {len(files_varying_lengthscale)} files")
print(f"Found {len(files_different_depth)} files")
#print(f"Found {len(files_4_op_uniform_3)} files")

#print(f"Found {len(files_4_op_uniform_jz)} files")

# get all the results in a dataframe
results = []
for files, xp_name in zip([files_6_op, files_4_op, files_4_op_mlm, files_4_op_small_model, files_4_op_uniform, files_unknown, files_varying_lengthscale, files_different_depth],
                           ["6_op", "4_op", "4_op_mlm", "4_op_small_model", "4_op_uniform", "unknown", "varying_lengthscale", "different_depth"]):
    for file in files:
        df = pd.read_csv(f"results/{file}")#.sample(10_000)
        df["experiment"] = xp_name
        results.append(df)

# concatenate the dataframes
results = pd.concat(results)

# remove missing columns
results = results.dropna(axis=1, how="all")

In [ ]:
results = results.fillna("missing")
# remove na epochs
results = results[results["epoch"] != "missing"]
results = results.applymap(str)
results["loss_mlm"] = results["loss_mlm"].astype(float)
results["loss_num"] = results["loss_num"].astype(float)
results["loss_total"] = results["loss_total"].astype(float)
results["val_loss_mlm"] = results["val_loss_mlm"].astype(float)
results["val_loss_num"] = results["val_loss_num"].astype(float)
results["val_loss_total"] = results["val_loss_total"].astype(float)
results["epoch"] = results["epoch"].astype(float).astype(int)

In [ ]:
import plotly.express as px
import numpy as np

xp_fancy_names = {
    "6_op": "6 operands (predicting RHS)",
    "4_op": "4 operands (predicting RHS)",
    "4_op_mlm": "4 operands (predicting all tokens)",
    "4_op_small_model": "4 operands (small model)",
    "4_op_uniform": "4 operands (uniform)",
    "unknown": "unknown",
    "varying_lengthscale": "4 operands (varying lengthscale)",
}

xp_name = "unknown"
# filter
#results_to_plot = results[results["experiment"] == xp_name]
results_to_plot = results.copy()

# split dataset path to take only the part between /
results_to_plot["dataset_path"] = results_to_plot["dataset_path"].str.split("/").str[2]
results_to_plot["collator"] = results_to_plot["collator"].str.split(".").str[0]

results_to_plot["epoch"] = results_to_plot["epoch"].astype(float)
filter_dic = {
    "num_layers": "6",
    "reverse_numerical_embedding": "False",
    "d_model": "384",
    "dataset_path": "tokenized_fad_4_op_uniform",
    "normalize_token_emb": "True",
    "tokenizer_path": "./data/tokenized_fad_bigtokenizer_xval.json",
    #"fourier_length_scale": 0.1,
    #"fourier_length_scale": "0.0520833333333333",
    #"gaussian_embedding_sd": "[0.052083333333333336]",
    #"gaussian_embedding_sd": 0.1,
    #"epoch": "30",
}

# allow missing or value in filter_dic
for key, value in filter_dic.items():
    results_to_plot = results_to_plot[np.isin(results_to_plot[key], [value, "missing"])]

# remove missing tokenizer_path
results_to_plot = results_to_plot[results_to_plot["tokenizer_path"] != "missing"]

# if embedding is fourier, only keep "fourier_length_scale": "0.0520833333333333",
results_to_plot = results_to_plot[(results_to_plot["embedding_type"] != "fourier") | (results_to_plot["fourier_length_scale"] == "0.0052083333333333")]
# same for gaussian with "[0.052083333333333336]"
#results_to_plot = results_to_plot[(results_to_plot["embedding_type"] != "gaussian") | (results_to_plot["gaussian_embedding_sd"] == "[0.06510416666666667]")]
results_to_plot = results_to_plot[(results_to_plot["embedding_type"] != "gaussian") | (results_to_plot["gaussian_embedding_sd"] == "[0.026041666666666668]")]


# remove duplicates
results_to_plot = results_to_plot.drop_duplicates()

# print model size
print("num_layers", results_to_plot["num_layers"].unique())
print("embedding_type", results_to_plot["embedding_type"].unique())
print("d_model", results_to_plot["d_model"].unique())
print("nhead", results_to_plot["nhead"].unique())
print("dim_feedforward", results_to_plot["dim_feedforward"].unique())
print("dataset_path", results_to_plot["dataset_path"].unique())
print("fourier_length_scale", results_to_plot["fourier_length_scale"].unique())
print("gaussian_embedding_sd", results_to_plot["gaussian_embedding_sd"].unique())
# reverse_numerical_embedding
print("reverse_numerical_embedding", results_to_plot["reverse_numerical_embedding"].unique())
# collator
print("collator", results_to_plot["collator"].unique())
print("normalize_token_emb", results_to_plot["normalize_token_emb"].unique())
print("nomalize_pos_enc", results_to_plot["normalize_pos_enc"].unique())


# fix the palette for the colors
palette = px.colors.qualitative.Plotly
palette = np.array(palette)

# remove gaussian embedding
#results_to_plot = results_to_plot[results_to_plot["embedding_type"] != "gaussian"]


#group_cols = ["epoch", "dim_feedforward", "dropout", "norm_first", "embedding_type"]
group_cols = results_to_plot.columns
#group_cols.remove(["loss_mlm", "loss_num", "loss_total", "iter"])
group_cols = [col for col in group_cols if col not in ["loss_mlm", "loss_num", "loss_total", "val_loss_mlm", "val_loss_num", "val_loss_total", "epoch"]]
import pandas as pd


# Define the threshold
threshold = 0.0005

# Group by the specified columns and filter
def min_epoch_below_threshold(group):
    below_threshold = group[group['val_loss_total'] < threshold]
    if not below_threshold.empty:
        return below_threshold['epoch'].min()
    return None  # Return None if no epochs are below the threshold

# Group by the specified columns and apply the function
result = results_to_plot.groupby(group_cols).apply(min_epoch_below_threshold).reset_index()

# Rename the result column for clarity
result.rename(columns={0: 'min_epoch_to_reach_threshold'}, inplace=True)

# Display the result
print(result)

threshold = 0.001
# Group by the specified columns and filter
result2 = results_to_plot.groupby(group_cols).apply(lambda group: group[group['val_loss_total'] < threshold]['epoch'].min()).reset_index()

# Rename the result column for clarity
result2.rename(columns={0: 'min_epoch_to_reach_threshold'}, inplace=True)

threshold = 0.0001
# Group by the specified columns and filter
result3 = results_to_plot.groupby(group_cols).apply(lambda group: group[group['val_loss_total'] < threshold]['epoch'].min()).reset_index()

# Rename the result column for clarity
result3.rename(columns={0: 'min_epoch_to_reach_threshold'}, inplace=True)

# Display the result
#fig = px.scatter(result, x='min_epoch_to_reach_threshold', y="embedding_type", title='Minimum epoch to reach threshold', height=900, color='min_epoch_to_reach_threshold', facet_col='dataset_path',
#                 hover_data=result.columns)
#fig.show()
#results_to_plot[results_to_plot["epoch"] == 0.0]
# combine the two results
import pandas as pd



# Ensure 'group_cols' are set correctly as per your earlier code
group_cols = results_to_plot.columns
group_cols = [col for col in group_cols if col not in ["loss_mlm", "loss_num", "loss_total", "val_loss_mlm", "val_loss_num", "val_loss_total", "epoch", "min_epoch_to_reach_threshold"]]

# Merge results and results2 on group columns
merged_results = pd.merge(result, result2, on=group_cols, how="left", suffixes=('_threshold_0.0005', '_threshold_0.001'))
merged_results = pd.merge(merged_results, result3, on=group_cols, how="left")
# rename min_epoch_to_reach_threshold to min_epoch_to_reach_threshold_threshold_0.0001
merged_results.rename(columns={"min_epoch_to_reach_threshold": "min_epoch_to_reach_threshold_threshold_0.0001"}, inplace=True)
# reorder columns
merged_results = merged_results[group_cols + ['min_epoch_to_reach_threshold_threshold_0.001', 'min_epoch_to_reach_threshold_threshold_0.0005', 'min_epoch_to_reach_threshold_threshold_0.0001']]

# Calculate standard deviation, min, and max for the epoch columns
import pandas as pd

# Assuming merged_results is already defined and contains the necessary columns

# Calculate mean, std, min, and max grouped by 'embedding_type'
grouped_stats = merged_results.groupby('embedding_type').agg({
    'min_epoch_to_reach_threshold_threshold_0.0005': ['mean', 'std', 'min', 'max', "count"],
    'min_epoch_to_reach_threshold_threshold_0.001': ['mean', 'std', 'min', 'max'],
    'min_epoch_to_reach_threshold_threshold_0.0001': ['mean', 'std', 'min', 'max'],
}).reset_index()

# Flatten the MultiIndex in columns
grouped_stats.columns = ['_'.join(col).strip() if col[1] else col[0] for col in grouped_stats.columns.values]

# Format columns as 'mean (min-max)'
grouped_stats['Formatted_0.0005'] = grouped_stats.apply(
    lambda row: f"{row['min_epoch_to_reach_threshold_threshold_0.0005_mean']:.2f} ({row['min_epoch_to_reach_threshold_threshold_0.0005_min']}-{row['min_epoch_to_reach_threshold_threshold_0.0005_max']})",
    axis=1
)
grouped_stats['Formatted_0.001'] = grouped_stats.apply(
    lambda row: f"{row['min_epoch_to_reach_threshold_threshold_0.001_mean']:.2f} ({row['min_epoch_to_reach_threshold_threshold_0.001_min']}-{row['min_epoch_to_reach_threshold_threshold_0.001_max']})",
    axis=1
)
grouped_stats['Formatted_0.0001'] = grouped_stats.apply(
    lambda row: f"{row['min_epoch_to_reach_threshold_threshold_0.0001_mean']:.2f} ({row['min_epoch_to_reach_threshold_threshold_0.0001_min']}-{row['min_epoch_to_reach_threshold_threshold_0.0001_max']})",
    axis=1
)

# Select the final columns for the table
final_table = grouped_stats[['embedding_type', 'Formatted_0.001', 'Formatted_0.0005', 'Formatted_0.0001']]
#final_table.columns = ['Group', 'Min Epoch (Threshold 0.0005)', 'Min Epoch (Threshold 0.001)']

final_table.columns = ['Embedding', 'RMSE=0.001', 'RMSE=0.0005', 'RMSE=0.0001']
# replace nans by "Not reached" (and don't show the min-max in this case)
#final_table = final_table.fillna("Not reached")
final_table = final_table.replace("nan (nan-nan)", "Not reached")

# Convert DataFrame to LaTeX
latex_table = final_table.to_latex(index=False, caption="Minimum Epochs to Reach Thresholds")

# Add these as new columns to the DataFrame
# merged_results['Std Dev (Threshold 0.0005)'] = std_dev[0]
# merged_results['Min-Max (Threshold 0.0005)'] = f"{min_values[0]} - {max_values[0]}"
# merged_results['Std Dev (Threshold 0.001)'] = std_dev[1]
# merged_results['Min-Max (Threshold 0.001)'] = f"{min_values[1]} - {max_values[1]}"


# Select and rename columns for clarity
#final_table = merged_results[['embedding_type', 'min_epoch_to_reach_threshold_threshold_0.0005', 'Std Dev (Threshold 0.0005)', 'Min-Max (Threshold 0.0005)', 
#                              'min_epoch_to_reach_threshold_threshold_0.001', 'Std Dev (Threshold 0.001)', 'Min-Max (Threshold 0.001)']]
#final_table.columns = ['Group', 'Min Epoch (Threshold 0.0005)', 'Std Dev (0.0005)', 'Range (0.0005)', 'Min Epoch (Threshold 0.001)', 'Std Dev (0.001)', 'Range (0.001)']
# Select and rename columns for clarity
# final_table = merged_results[['embedding_type', 'min_epoch_to_reach_threshold_threshold_0.0005', 'min_epoch_to_reach_threshold_threshold_0.001']]
# final_table.columns = ['Group', 'Min Epoch (Threshold 0.0005)', 'Min Epoch (Threshold 0.001)']

# # # Convert DataFrame to LaTeX
# latex_table = final_table.to_latex(index=False, caption="Minimum Epochs to Reach Thresholds", label="tab:thresholds", longtable=True)
# print(latex_table)




In [ ]:
np.unique(results_to_plot[(results_to_plot["epoch"] == 1.) & (results_to_plot["embedding_type"] == "gaussian")]["gaussian_embedding_sd"], return_counts=True)

In [ ]:
np.unique(results_to_plot[(results_to_plot["epoch"] == 1) & (results_to_plot["embedding_type"] == "fourier")]["fourier_length_scale"], return_counts=True)

In [ ]:
results_to_plot

In [ ]:
print(latex_table)

In [ ]:
results_to_plot[(results_to_plot["iter"] == "2") & (results_to_plot["epoch"] == 10)]

In [ ]:
grouped_stats

In [ ]:
px.scatter(results_to_plot, x="epoch", y="val_loss_total", color="embedding_type", title="Validation loss per epoch", facet_col="iter")

In [ ]:
)